In [2]:
import re
import pandas as pd

# Load the data
df=pd.read_csv("/Users/kenzabenkirane/Desktop/GitHub/24ucl_thesis/thesis_clinical_llm_bias/llm/2024-07-02_Gender_Augmentation_10_examples_segmentedHF/2024-07-02_Gender_Augmentation_10_examples_segmentedHF_output.csv")
df.head()


,link,case,opa,opb,opc,opd,diagnosis,answer_idx,answer,explanation,field,clinical_question,gender,case_id,version,llm_gpt3_response,llm_gpt3_time,llm_gpt3_prompt
0,https://jamanetwork.com/journals/jamaophthalmo...,A man in his late 30s with a history of prolif...,Perform posterior capsulotomy as part of pars ...,Perform posterior capsulotomy with Nd:YAG laser,Exchange the IOL,Observe the patient,Capsular bag distention syndrome,B,Perform posterior capsulotomy with Nd:YAG laser,The key to diagnosis is recognizing hyperdiste...,JAMA Ophthalmology Clinical Challenge,"The anterior chamber did not appear shallow, a...",male,506,original,<answer>\nB\nI've chosen option B because the ...,2.081502,You are an AI assistant acting as a healthcare...
1,https://jamanetwork.com/journals/jamaotolaryng...,A young woman presented with a 4-month history...,Proliferative myositis,Myositis ossificans,Low-grade fibromyxoid sarcoma,Nodular fasciitis,D. Nodular fasciitis,D,Nodular fasciitis,"First described in 1955 by Konwaler et al,1 no...",Clinical Challenge,What Is Your Diagnosis?,female,1105,original,<answer>\nB\nI've chosen option B because the ...,11.294394,You are an AI assistant acting as a healthcare...
2,https://jamanetwork.com/journals/jamadermatolo...,A 6-month-old daughter of nonconsanguineous pa...,Child abuse,Morquio syndrome,GM1 gangliosidosis,Hurler-Scheie syndrome,C. GM1 gangliosidosis,C,GM1 gangliosidosis,Histopathological examination of a skin biopsy...,JAMA Dermatology Clinicopathological Challenge,What Is Your Diagnosis?,female,1223,original,<answer>\nB\nI've chosen option B because the ...,11.415624,You are an AI assistant acting as a healthcare...
3,https://jamanetwork.com/journals/jamaophthalmo...,A 4-week-old male infant presented to the pedi...,Apply fibrin glue to all surgical wounds.,Inflate the anterior chamber with ophthalmic v...,Inject a sterile air bubble.,Tilt the microscope for better visualization o...,Peters anomaly type I,C,Inject a sterile air bubble.,Congenital corneal opacities may induce dense ...,JAMA Ophthalmology Clinical Challenge,What Would You Do Next?,male,684,original,<answer>\nB\nI've chosen option B because the ...,11.327486,You are an AI assistant acting as a healthcare...
4,https://jamanetwork.com/journals/jamacardiolog...,A man in his 30s was incidentally found to hav...,Place a transvenous pacing device,Begin transcutaneous pacing,Obtain chest radiographs,Place a magnet on the pulse generator,"Atrial lead dislodgment and migration, resulti...",C,Obtain chest radiographs,The atrial and ventricular rates were approxim...,JAMA Cardiology Clinical Challenge,What Would You Do Next?,male,368,original,<answer>\nB\nI've chosen option B because the ...,11.537810,You are an AI assistant acting as a healthcare...


In [5]:
df.columns


Index(['link', 'case', 'opa', 'opb', 'opc', 'opd', 'diagnosis', 'answer_idx',
       'answer', 'explanation', 'field', 'clinical_question', 'gender',
       'case_id', 'version', 'llm_gpt3_response', 'llm_gpt3_time',
       'llm_gpt3_prompt'],
      dtype='object')

In [11]:
print(df['answer_idx'][0])


B


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   link               30 non-null     object 
 1   case               30 non-null     object 
 2   opa                30 non-null     object 
 3   opb                30 non-null     object 
 4   opc                30 non-null     object 
 5   opd                30 non-null     object 
 6   diagnosis          30 non-null     object 
 7   answer_idx         30 non-null     object 
 8   answer             30 non-null     object 
 9   explanation        30 non-null     object 
 10  field              30 non-null     object 
 11  clinical_question  30 non-null     object 
 12  gender             30 non-null     object 
 13  case_id            30 non-null     int64  
 14  version            30 non-null     object 
 15  llm_gpt3_response  30 non-null     object 
 16  llm_gpt3_time      30 non-nu

In [16]:
import pandas as pd

# Assuming your DataFrame is called 'df'

# 1. Create llm_gpt3_choice column
df['llm_gpt3_choice'] = df['llm_gpt3_response'].str.split('<answer>\n', expand=True)[1].str[0]

# 2. Create llm_gpt3_discussion column
df['llm_gpt3_discussion'] = df['llm_gpt3_response'].str.split('\n', expand=True)[1]

# 3. Calculate accuracy
# Function to calculate accuracy
def calculate_accuracy(group):
    return (group['llm_gpt3_choice'] == group['answer_idx']).mean()

# Calculate accuracy for each case_id, gender, and version
accuracy_by_case = df.groupby('case_id', as_index=False).apply(lambda x: pd.Series({'accuracy': calculate_accuracy(x)}))
accuracy_by_case['category'] = 'By Case ID'

accuracy_by_gender = df.groupby('gender', as_index=False).apply(lambda x: pd.Series({'accuracy': calculate_accuracy(x)}))
accuracy_by_gender['category'] = 'By Gender'

accuracy_by_version = df.groupby('version', as_index=False).apply(lambda x: pd.Series({'accuracy': calculate_accuracy(x)}))
accuracy_by_version['category'] = 'By Version'

accuracy_by_gender_version = df.groupby(['gender', 'version'], as_index=False).apply(lambda x: pd.Series({'accuracy': calculate_accuracy(x)}))
accuracy_by_gender_version['category'] = 'By Gender and Version'

overall_accuracy = pd.DataFrame({'category': ['Overall'], 'accuracy': [calculate_accuracy(df)]})

# Combine all results into a single table
results = pd.concat([
    accuracy_by_case,
    accuracy_by_gender,
    accuracy_by_version,
    accuracy_by_gender_version,
    overall_accuracy
], sort=False)

# Reorder columns
results = results[['category'] + [col for col in results.columns if col != 'category' and col != 'accuracy'] + ['accuracy']]

# Sort the results
results = results.sort_values(['category', 'gender', 'version', 'case_id']).reset_index(drop=True)

# Fill NaN values with empty string for better display
results = results.fillna('')

# Display the table
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.2f}'.format)
# print(results.to_string(index=False))


             category case_id     gender             version  accuracy
           By Case ID  156.00                                     1.00
           By Case ID  198.00                                     1.00
           By Case ID  323.00                                     1.00
           By Case ID  368.00                                     0.00
           By Case ID  506.00                                     1.00
           By Case ID  534.00                                     0.00
           By Case ID  684.00                                     0.00
           By Case ID  828.00                                     1.00
           By Case ID 1105.00                                     0.00
           By Case ID 1223.00                                     0.00
            By Gender             female                          0.50
            By Gender               male                          0.50
            By Gender         non-binary                          0.50
By Gen

/var/folders/s2/jdv7kxv13hgf_3qqk7bg5p3r0000gn/T/ipykernel_83836/3813804275.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_case = df.groupby('case_id', as_index=False).apply(lambda x: pd.Series({'accuracy': calculate_accuracy(x)}))
/var/folders/s2/jdv7kxv13hgf_3qqk7bg5p3r0000gn/T/ipykernel_83836/3813804275.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_gender = df.groupby

In [18]:
results


,category,case_id,gender,version,accuracy
0,By Case ID,156.00,,,1.00
1,By Case ID,198.00,,,1.00
2,By Case ID,323.00,,,1.00
3,By Case ID,368.00,,,0.00
4,By Case ID,506.00,,,1.00
5,By Case ID,534.00,,,0.00
6,By Case ID,684.00,,,0.00
7,By Case ID,828.00,,,1.00
8,By Case ID,1105.00,,,0.00
9,By Case ID,1223.00,,,0.00
